<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Spliting-Dataset" data-toc-modified-id="Spliting-Dataset-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Spliting Dataset</a></span></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Modeling</a></span></li><li><span><a href="#Predict" data-toc-modified-id="Predict-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Predict</a></span></li><li><span><a href="#Recommendation" data-toc-modified-id="Recommendation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Recommendation</a></span></li></ul></div>

In [1]:
import os
import random
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google_drive_downloader import GoogleDriveDownloader as gdd
from sklearn.model_selection import train_test_split
import copy
import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Concatenate, Add, Dot, Multiply, Reshape, Activation, BatchNormalization, SimpleRNNCell, RNN, SimpleRNN, LSTM, Embedding, Bidirectional, TimeDistributed, Conv1D, Conv2D, MaxPool1D, MaxPool2D, GlobalMaxPool1D, GlobalMaxPool2D, AveragePooling1D, AveragePooling2D, GlobalAveragePooling1D, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import SGD, Adam, Adagrad
from tensorflow.keras.metrics import RootMeanSquaredError, BinaryCrossentropy, SparseCategoricalAccuracy
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.activations import linear, sigmoid, relu

tqdm.pandas()
np.set_printoptions(precision=3)
plt.style.use("dark_background")

C:\Users\82104\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [116]:
movies = pd.read_csv("./Datasets/MovieLens/movies.csv", index_col=0)
genres = pd.read_csv("./Datasets/MovieLens/genres.csv", index_col=0)
gdd.download_file_from_google_drive(file_id="15vsm-VWAC3Y-7jr7ROL_xy0ufkIfqSke", dest_path="D:/ratings.csv")
ratings = pd.read_csv("D:/ratings.csv")
ratings = ratings.drop(["Unnamed: 0"], axis=1)

ratings = ratings.drop(["rated_at"], axis=1)

In [42]:
likes = ratings[(ratings["rating"]>=4.0)]
dislikes = ratings[(ratings["rating"]<=2.0)]

user_n_ratings = likes["user_id"].value_counts()
cumsum_user = user_n_ratings.cumsum()/len(likes)
concat = pd.concat([user_n_ratings, cumsum_user], axis=1)
concat.columns = ["n_rating", "cumsum"]
# 평점을 `thr_user`개 이상 남긴 사용자가 전체 평점의 70%를 차지합니다.
thr_user = concat[concat["cumsum"]<0.4]["n_rating"].tolist()[-1]

item_n_ratings = likes["movie_id"].value_counts()
cumsum_item = item_n_ratings.cumsum()/len(likes)
concat = pd.concat([item_n_ratings, cumsum_item], axis=1)
concat.columns = ["n_rating", "cumsum"]
# 평점이 `thr_item`개 이상인 영화가 전체 평점의 70%를 차지합니다.
thr_item = concat[concat["cumsum"]<0.4]["n_rating"].tolist()[-1]

likes_tar = likes[(likes["user_id"].isin(user_n_ratings[user_n_ratings>=thr_user].index)) & (likes["movie_id"].isin(item_n_ratings[item_n_ratings>=thr_item].index))]

print(f"len(likes): {len(likes):,}")
print(f"len(likes_tar): {len(likes_tar):,}")
print(f"{len(likes_tar)/len(likes):.1%}")

len(likes): 9,995,410
len(likes_tar): 1,165,926
11.7%


In [43]:
movie_n_ratings = ratings.groupby(["movie_id"]).size().sort_values(ascending=False)

# user_movies = ratings.groupby(["user_id"])["movie_id"].apply(frozenset)
# X_te["items"] = X_te["user_id"].apply(lambda x:user_movies[x])
user_dislikes = dislikes.groupby(["user_id"])["movie_id"].progress_apply(list)
likes_tar["neg_movie_id"] = likes_tar["user_id"].progress_apply(lambda x:int(random.choices(user_dislikes[x], k=1, weights=movie_n_ratings[user_dislikes[x]])[0]) if x in user_dislikes else np.nan)
likes_tar = likes_tar[likes_tar["neg_movie_id"].notna()]

<ipython-input-43-e557f06183e9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  likes_tar["neg_movie_id"] = likes_tar["user_id"].progress_apply(lambda x:random.choices(user_dislikes[x], k=1, weights=movie_n_ratings[user_dislikes[x]])[0] if x in user_dislikes else np.nan)


In [ ]:
# y_te = likes_tar_te[["rating"]]

# X_te = copy.deepcopy(likes_tar_te)
# X_te = X_te.drop(["rating"], axis=1)


# `movie_id`: 높은 평점을 준 영화 1개
# `items_100`: 낮은 평점을 준 영화 100개
# X_te["items_100"] = X_te.progress_apply(lambda x:random.choices(list(x["items"] - {x["movie_id"]}), k=100, weights=movie_n_ratings[list(x["items"] - {x["movie_id"]})]), axis=1)

# Spliting Dataset

In [49]:
trs = list()
tes = list()
for _, group in likes_tar.groupby(["user_id"]):
    # 최근에 평점을 남긴 영화가 Test data set에 포함되도록 합니다.
    tr, te = train_test_split(group, test_size=0.2, shuffle=False)
    trs.append(tr)
    tes.append(te)
likes_tar_tr = pd.concat(trs, axis=0)
likes_tar_te = pd.concat(tes, axis=0)
likes_tar_tr = likes_tar_tr.sample(frac=1)
likes_tar_te = likes_tar_te.sample(frac=1)

print(f"len(likes_tar_tr): {len(likes_tar_tr):,}")
print(f"len(likes_tar_te): {len(likes_tar_te):,}")


len(likes_tar_tr): 917,783
len(likes_tar_te): 235,710


In [23]:
# user_movies = ratings.groupby(["user_id"])["movie_id"].apply(frozenset)
# X_te["items"] = X_te["user_id"].apply(lambda x:user_movies[x])
# user_dislikes = dislikes.groupby(["user_id"])["movie_id"].apply(frozenset)
# X_te["dislikes"] = X_te["user_id"].apply(lambda x:user_dislikes[x] if x in user_dislikes else np.nan)

# y_te = likes_tar_te[["rating"]]

# X_te = copy.deepcopy(likes_tar_te)
# X_te = X_te.drop(["rating"], axis=1)

# movie_n_ratings = likes_tar.groupby(["movie_id"]).size().sort_values(ascending=False)

# `movie_id`: 높은 평점을 준 영화 1개
# `items_100`: 낮은 평점을 준 영화 100개
# X_te["items_100"] = X_te.progress_apply(lambda x:random.choices(list(x["items"] - {x["movie_id"]}), k=100, weights=movie_n_ratings[list(x["items"] - {x["movie_id"]})]), axis=1)

# Modeling

In [58]:
input_user = Input(shape=(), name="Input_user")
input_pos = Input(shape=(), name="Input_pos")
input_neg = Input(shape=(), name="Input_neg")
inputs = [input_user, input_pos, input_neg]

n_users = ratings["user_id"].max() + 1
n_items = ratings["movie_id"].max() + 1
dim = 64
embedding_user = Embedding(input_dim=n_users, output_dim=dim, name="Embedding_user")
embedding_item = Embedding(input_dim=n_items, output_dim=dim, name="Embedding_item")

z1 = embedding_user(input_user)
z2 = embedding_item(input_pos)
z3 = embedding_item(input_neg)

pos_score = Dot(axes=(1, 1))([z1, z2])
neg_score = Dot(axes=(1, 1))([z1, z3])
diff = pos_score - neg_score
outputs = sigmoid(diff)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=Adagrad(), loss="binary_crossentropy", metrics=["binary_accuracy"])

X = [likes_tar_tr["user_id"].values, likes_tar_tr["movie_id"].values, likes_tar_tr["neg_movie_id"].values]
y = np.ones(shape=(len(likes_tar_tr), 1))

es = EarlyStopping(monitor="val_loss", mode="auto", verbose=1, patience=2)
model_path = "./movielens_bpr_tf.h5"
mc = ModelCheckpoint(filepath=model_path, monitor="val_binary_accuracy", mode="auto", verbose=1, save_best_only=True)
model.fit(x=X, y=y, validation_split=0.2, batch_size=128, epochs=32, verbose=2, callbacks=[es, mc])

# Predict

In [93]:
model_test = Model(inputs=[input_user, input_pos], outputs=pos_score)

In [95]:
likes_tar_te.head()

,user_id,movie_id,rating,neg_movie_id
15055150,103973,7438,4.5,434
5046580,34632,4995,5.0,1127
17471603,120842,6874,5.0,76251
2644634,17961,8961,4.0,6957
16507849,114204,2804,5.0,10


In [ ]:
for name, row in likes_tar_te.iterrows():
    input_user = np.array([row["user_id"]])
    input_pos = np.array([row["movie_id"]])
    pos_score = model_test.predict([input_user, input_pos])[0][0]
    
    input_neg = np.array([row["neg_movie_id"]])
    neg_score = model_test.predict([input_user, input_neg])[0][0]

# Recommendation

In [130]:
item_embs = pd.DataFrame(model.get_layer("Embedding_item").get_weights()[0])
item_embs = item_embs[item_embs.index.isin(movies["id"])]
id2title = {row["id"]:row["title"] for name, row in movies.iterrows()}
item_embs.index = item_embs.apply(lambda x:id2title[x.name], axis=1)

In [177]:
movies[movies["title"].str.contains("Monsters,")]

,id,title,release_year
4790,4886,"Monsters, Inc.",2001


In [179]:
# tar = "Star Wars: Episode V - The Empire Strikes Back"
tar = "Monsters, Inc."
tar_vec = item_embs.loc[tar]
rank = item_embs.dot(tar_vec).sort_values(ascending=False)[:10]

rank

Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark)    1.951831
Star Wars: Episode V - The Empire Strikes Back                             1.853958
Shawshank Redemption, The                                                  1.842067
Star Wars: Episode IV - A New Hope                                         1.756639
Usual Suspects, The                                                        1.735136
Godfather, The                                                             1.730903
Back to the Future                                                         1.722280
One Flew Over the Cuckoo's Nest                                            1.667557
Silence of the Lambs, The                                                  1.652493
Goodfellas                                                                 1.643711
dtype: float32